In [137]:
import numpy as np
import mir_eval
import collections
import json
import os, os.path
import csv
from statistics import mean

In [112]:
# load data
notes_path = 'Notes'
results_path = 'Results'
songtitle = 'Kumushki_last7'
transcriber1 = 'PP'
transcriber2 = 'OV'
notes_prefix = os.path.join(notes_path, 'notes_')
filename = 'Notes/notes_Kumushki_last7_'
notes_filename1 = notes_prefix + songtitle + '_' + transcriber1 + '.csv'
notes_filename2 = notes_prefix + songtitle + '_' + transcriber2 + '.csv'
notes_filenameAUTO = notes_prefix + songtitle + '_' + 'AUTO' + '.csv'
if not os.path.isdir(results_path):
    os.mkdir(results_path)

In [113]:
def load_tony_notes(filename):
    tony_notes = np.genfromtxt(filename, delimiter=',', skip_header=0)
    onsets = tony_notes[:, 0]
    pitches = tony_notes[:, 1]
    durations = tony_notes[:, 2]
    if min(durations) == 0:
        for idx in range(len(duration)):
            if durations[idx] == 0:
                durations[idx] = 0.00001
    offsets = np.sum([onsets, durations],axis=0)
    if onsets[0] < 0:
        onsets[0] = 0
    if offsets[0] <= 0:
        offsets[0] = 0.000001
    segments = np.array(list(zip(onsets, offsets)))
    return segments, pitches

In [153]:
def compare_two_transcriptions(notes_filename1, notes_filename2):
    segments1, pitches1 = load_tony_notes(notes_filename1)
    segments2, pitches2 = load_tony_notes(notes_filename2)
    mir_eval.transcription.validate(segments1, pitches1, segments2, pitches2)
    result_dict = mir_eval.transcription.evaluate(segments1, pitches1,  segments2, pitches2)
    return result_dict

In [ ]:
result_dict = compare_two_transcriptions(notes_filename1, notes_filename2)
result_dict

In [116]:
# export json
json_outfilename = os.path.join(results_path, songtitle + '_' + transcriber1 + '_' + transcriber2 + '.json')
with open(outfilename, 'w+') as outfile:
    json.dump(result_dict, outfile, indent=4)

In [123]:
# export csv
csv_outfilename = os.path.join(results_path, songtitle + '_' + transcriber1 + '_' + transcriber2 + '.csv')
with open(csv_outfilename, 'w') as csvfile:
    csvwriter = csv.writer(csvfile)
    for key in result_dict.keys():
        csvwriter.writerow([key, result_dict[key]])

In [158]:
# compare two transcriptions and auto
def compare_2human_vs_auto(notes_filename1, notes_filename2, notes_filenameAUTO):
    result_h1h2 = compare_two_transcriptions(notes_filename1, notes_filename2)
    result_h1auto = compare_two_transcriptions(notes_filename1, notes_filenameAUTO)
    result_h2auto = compare_two_transcriptions(notes_filename2, notes_filenameAUTO)
    csv_outfilename = os.path.join(results_path, songtitle + '_' + transcriber1 + '_' + transcriber2 + '_' + 'AUTO' + '.csv')
    with open(csv_outfilename, 'w') as csvfile:
        csvwriter = csv.writer(csvfile)
        csvwriter.writerow(['',transcriber1 + ' vs ' + transcriber2, transcriber1 + ' vs AUTO', transcriber2 + ' vs AUTO', 'mean H vs AUTO'])
        for key in result_h1h2.keys():
            csvwriter.writerow([key, round(result_h1h2[key],2), round(result_h1auto[key],2), round(result_h2auto[key],2), round(mean((result_h1auto[key], result_h2auto[key])),2)])

In [159]:
# quantitative analysis for all songs in a folder
# each song should have three note files: for each transcriber and automatic
# of the form: notes_Kumushki_last7_AUTO.csv
# results are saved for each song in the Results/ folder

songlist = []
for root, dirs, files in os.walk(notes_path):
    for filename in files:
        if filename.startswith('notes'):
            songtitle = filename.replace('notes_', '')
            songtitle = songtitle.replace('.csv', '')
            songtitle = songtitle.rsplit('_', 1)[0]
            if not songtitle in songlist:
                songlist.append(songtitle)

                notes_filename1 = notes_prefix + songtitle + '_' + transcriber1 + '.csv'
                notes_filename2 = notes_prefix + songtitle + '_' + transcriber2 + '.csv'
                notes_filenameAUTO = notes_prefix + songtitle + '_' + 'AUTO' + '.csv'
                if not os.path.isdir(results_path):
                    os.mkdir(results_path)

                compare_2human_vs_auto(notes_filename1, notes_filename2, notes_filenameAUTO)

            


Lazar10-12_manseg
Po zoriushke track 10
Lazar1-13
Po zoriushke track 04
Po zoriushke track 13
Kumushki_last7
Po zoriushke track 07
